# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f368cd952e0>
├── 'a' --> tensor([[ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824]])
└── 'x' --> <FastTreeValue 0x7f368cd95070>
    └── 'c' --> tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                        [ 1.3962, -0.5990,  0.0647, -0.1122],
                        [-1.0775,  0.1098, -1.5331,  0.5423]])

In [4]:
t.a

tensor([[ 1.4802, -0.5345, -0.6261],
        [-0.0249,  0.3292, -0.3824]])

In [5]:
%timeit t.a

63.8 ns ± 0.0507 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f368cd952e0>
├── 'a' --> tensor([[-0.7819, -1.0343,  1.2912],
│                   [-1.1933, -0.4996,  1.1648]])
└── 'x' --> <FastTreeValue 0x7f368cd95070>
    └── 'c' --> tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                        [ 1.3962, -0.5990,  0.0647, -0.1122],
                        [-1.0775,  0.1098, -1.5331,  0.5423]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.0858 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4802, -0.5345, -0.6261],
               [-0.0249,  0.3292, -0.3824]]),
    x: Batch(
           c: tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4802, -0.5345, -0.6261],
        [-0.0249,  0.3292, -0.3824]])

In [11]:
%timeit b.a

58.1 ns ± 0.111 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2638,  0.9770,  1.8512],
               [ 0.8206, -0.1424,  0.9728]]),
    x: Batch(
           c: tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 5.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

821 ns ± 0.324 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 34.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f368cd9b430>
├── 'a' --> tensor([[[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]],
│           
│                   [[ 1.4802, -0.5345, -0.6261],
│                    [-0.0249,  0.3292, -0.3824]]])
└── 'x' --> <FastTreeValue 0x7f35dfa0c3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 84.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f35dfa74fa0>
├── 'a' --> tensor([[ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824],
│                   [ 1.4802, -0.5345, -0.6261],
│                   [-0.0249,  0.3292, -0.3824]])
└── 'x' --> <FastTreeValue 0x7f35dfa0cb80>
    └── 'c' --> tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                        [ 1.3962, -0.5990,  0.0647, -0.1122],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 33.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4200,  0.2607, -1.0204,  0.5231],
                       [ 1.3962, -0.5990,  0.0647, -0.1122],
                       [-1.0775,  0.1098, -1.5331,  0.5423]],
              
                      [[ 0.4200,  0.2607, -1.0204,  0.5231],
                       [ 1.3962, -0.5990,  0.0647, -0.1122],
                       [-1.0775,  0.1098, -1.5331,  0.5423]],
              
                      [[ 0.4200,  0.2607, -1.0204,  0.5231],
                       [ 1.3962, -0.5990,  0.0647, -0.1122],
                       [-1.0775,  0.1098, -1.5331,  0.5423]],
              
                      [[ 0.4200,  0.2607, -1.0204,  0.5231],
                       [ 1.3962, -0.5990,  0.0647, -0.1122],
                       [-1.0775,  0.1098, -1.5331,  0.5423]],
              
                      [[ 0.4200,  0.2607, -1.0204,  0.5231],
                       [ 1.3962, -0.5990,  0.0647, -0.1122],
                       [-1.0775,  0.1098, -1.5331,  0.5423]],

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423],
                      [ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423],
                      [ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423],
                      [ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423],
                      [ 0.4200,  0.2607, -1.0204,  0.5231],
                      [ 1.3962, -0.5990,  0.0647, -0.1122],
                      [-1.0775,  0.1098, -1.5331,  0.5423],
                      [ 0.4200,  0.2607, -1.0204,  0.5231],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 849 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
